In [14]:
import numpy as np

def export_counts(myfile, contrast, N):
    f = open(myfile, 'r')
    lines = f.readlines()
    result = []
    for x in lines:
        result.append(x.split('#')[0])
    f.close()
    a = [x for x in result if x != '']

    evolution_time = np.zeros(len(a))
    counts1 = np.zeros(len(a))
    error1 = np.zeros(len(a))
    counts2 = np.zeros(len(a))
    error2 = np.zeros(len(a))

    for i in range(len(a)):
        evolution_time[i]=np.asarray(a[i].split(), dtype=np.float32)[0]
        counts1[i] = np.asarray(a[i].split(), dtype=np.float32)[1]
        error1[i] = np.asarray(a[i].split(), dtype=np.float32)[3]
        counts2[i] = np.asarray(a[i].split(), dtype=np.float32)[2]
        error2[i] = np.asarray(a[i].split(), dtype=np.float32)[4]

    baseline = np.sum(counts2+counts1)/len(counts2)/2
    C0_up = baseline / (1 - 0.01 * contrast / 2)
    C0_down = C0_up * (1 - 0.01 * contrast)
    counts = counts2 - counts1

    T = evolution_time * 10 * N

    normalized_counts = (counts) / (C0_up - C0_down)
    freq=0.5/evolution_time/1.0e+6
    return normalized_counts, evolution_time, error1

In [20]:
my_files=['G:/data/QudiData/2018/07/20180706/PulsedMeasurement/20180706-1931-23_NV4_SpinLocking_0.1083_NV4_pulsed_measurement.dat',
          'G:/data/QudiData/2018/07/20180706/PulsedMeasurement/20180706-1652-37_NV4_SpinLocking_0.12_NV4_pulsed_measurement.dat'
         ]
N=16

In [21]:
normalized_counts, evolution_time, error= export_counts(my_files[0], 29.4, N)
normalized_counts2, evolution_time2, error2 = export_counts(my_files[1], 29.4, N)

In [23]:
from plotly import __version__
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Scatter

init_notebook_mode()

trace1 = Scatter(
    x = evolution_time*1e+6,
    y = normalized_counts,
    error_y=dict(
            type='data',
            array=error,
            visible=True
                ),
    mode = 'lines+markers',
    name = 'on resonance'
)

trace2 = Scatter(
    x = evolution_time2*1e+6,
    y = normalized_counts2,
    error_y=dict(
            type='data',
            array=error2,
            visible=True
                ),
    mode = 'lines+markers',
    name = 'off resonance'
)

trace3 = Scatter(
    x = evolution_time2*1e+6,
    y = np.abs(normalized_counts-normalized_counts2),
    error_y=dict(
            type='data',
            array=error2-error,
            visible=True
                ),
    mode = 'lines+markers',
    name = 'difference'
)


plot_data = [trace1, trace2, trace3]
plot(plot_data, filename='scatter-mode')

'file://C:\\Users\\pi3\\Documents\\GitHub\\qudi\\scatter-mode.html'

In [ ]:
np.savetxt('G:/data/QudiData/2018/01/20180119/PulsedMeasurement/counts_kddxy_6_pol.txt', normalized_counts, delimiter=',')
np.savetxt('G:/data/QudiData/2018/01/20180119/PulsedMeasurement/freq_kddxy_6_pol.txt', freq, delimiter=',')

In [ ]:
import pylab as plb
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

x=0.5/evolution_time/1.0e+6
y=normalized_counts

n = len(x)                          #the number of data
mean = sum(x*y)/n                   #note this correction
sigma = sum(y*(x-mean)**2)/n        #note this correction

def gaus(x,a,x0,sigma):
    return y[0]-a*exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus,x,y,p0=[0.3, 2.4, 0.1])

print(popt)

In [ ]:
from plotly import __version__
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Scatter

init_notebook_mode()

# print("plotly version:", __version__)

freq=0.5/evolution_time/1.0e+6
popt=[ 0.15,  2.41,  0.03]

trace1 = Scatter(
    x = freq,
    y = normalized_counts,
    mode = 'lines+markers',
    name = 'lines+markers'
)
trace2 = Scatter(
    x = np.linspace(freq.max(), freq.min(), 200),
    y = gaus(np.linspace(freq.max(), freq.min(), 200), *popt)-0.2,
    mode = 'lines',
    name = 'lines'
)

plot_data = [trace1, trace2]
plot(plot_data, filename='scatter-mode')

In [ ]:
from plotly import __version__
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Scatter

# t=np.linspace(0, 2*4*300e-9, 200)
# y=np.sin(2*np.pi*2.4*1e+6*t)

t=np.linspace(0, 4, 200)
y=np.exp(-t)

trace = Scatter(
    x = t,
    y = y,
    mode = 'lines',
    name = 'lines'
)

plot_data = [trace]
plot(plot_data, filename='scatter-mode')

In [40]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)
    
img=mpimg.imread('G:/Article/pictures/TOC_Graphic.png')
# plt.figure(figsize=cm2inch(8.251, 4.449))
plt.axis("off")
imgplot = plt.imshow(img)
plt.savefig('G:/Article/pictures/TOC_Graphic_300dpi.tiff', dpi = 300, figsize=cm2inch(8.251, 4.449))

In [42]:
from PIL import Image                                                                                
img = Image.open('G:/Article/pictures/TOC_Graphic.png')
img.show() 